# Причинно-следственный вывод для табличных данных

Причинно-следственный вывод включает в себя нахождение эффекта влияния для конкретного набора переменных, для другой переменной(м.б. с помощью изменения одной из переменных). Например, если A->B->C. Тогда все три переменные могут быть коррелированы, но вмешательство в C не влияет на значения B, поскольку C не является причинным предком B. Но, с другой стороны, вмешательства в A или B, оба влияют на значения C. 

Хотя существует много различных видов вопросов причинно-следственного вывода, которые могут заинтересовать человека, в настоящее время мы поддерживаем два вида - Средний эффект лечения (ATE) и условный ATE (CATE). В ATE мы воздействуем на один набор переменных с лечебным значением и контрольным значением и оцениваем ожидаемое изменение значения некоторой указанной целевой переменной. Математически,

$$\texttt{ATE} = \mathbb{E}[Y | \texttt{do}(X=x_t)] - \mathbb{E}[Y | \texttt{do}(X=x_c)]$$

где $\texttt{do}$ обозначает операцию вмешательства (оказания влияния на результат). Другими словами, ATE стремится определить относительную ожидаемую разницу в значении $Y$, когда мы вводим значение $X$, равное $x_t$, по сравнению с тем, когда мы вводим значение $X$, равное $x_c$. Здесь $x_t$ и $x_c$ являются соответственно значением обработки и контрольным значением.

CATE делает аналогичную оценку, но при некотором условии, заданном для набора переменных. Математически,

$$\texttt{CATE} = \mathbb{E}[Y | \texttt{do}(X=x_t), C=c] - \mathbb{E}[Y | \texttt{do}(X=x_c), C=c]$$

где мы ставим условие на некоторый набор переменных $C$, принимающих значение $c$. Обратите внимание здесь, что $X$ влияет на результат, а $C$ - нет.

In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import pickle as pkl
import time
from functools import partial

from causalai.data.data_generator import DataGenerator, ConditionalDataGenerator
from causalai.models.tabular.causal_inference import CausalInference
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

def define_treatments(name, t,c):
    treatment = dict(var_name=name,
                    treatment_value=t,
                    control_value=c)
    return treatment



## Непрерывные данные

### Средний лечебный эффект (ATE)
Для этого примера мы будем использовать синтетические данные, которые имеют линейную зависимость между переменными данных.

In [2]:
fn = lambda x:x
coef = 0.1
sem = {
        'a': [], 
        'b': [('a', coef, fn), ('f', coef, fn)], 
        'c': [('b', coef, fn), ('f', coef, fn)],
        'd': [('b', coef, fn), ('g', coef, fn)],
        'e': [('f', coef, fn)], 
        'f': [],
        'g': [],
        }
T = 1000
data, var_names, graph_gt = DataGenerator(sem, T=T, seed=0, discrete=False)

graph_gt

{'a': [],
 'b': ['a', 'f'],
 'c': ['b', 'f'],
 'd': ['b', 'g'],
 'e': ['f'],
 'f': [],
 'g': []}

In [3]:

# Notice c does not depend on a if we intervene on b. Hence intervening a has no effect in this case. 
# This can be verified by changing the intervention values of variable a, which should have no impact on the ATE. 
# (see graph_gt above)
t1='a' 
t2='b'
target = 'c'
target_var = var_names.index(target)

intervention11 = 10*np.ones(T)
intervention21 = 10*np.ones(T)
intervention_data1,_,_ = DataGenerator(sem, T=T, seed=0,
                        intervention={t1:intervention11, t2:intervention21})

intervention12 = -0.*np.ones(T)
intervention22 = -2.*np.ones(T)
intervention_data2,_,_ = DataGenerator(sem, T=T, seed=0,
                        intervention={t1:intervention12, t2:intervention22})



true_effect = (intervention_data1[:,target_var] - intervention_data2[:,target_var]).mean()
print("True ATE = %.2f" %true_effect)

True ATE = 1.20


In [4]:

tic = time.time()


treatments = [define_treatments(t1, intervention11,intervention12),\
             define_treatments(t2, intervention21,intervention22)]
# CausalInference_ = CausalInference(data, var_names, graph_gt,\
#         partial(MLPRegressor, hidden_layer_sizes=(100,100)) , False)
CausalInference_ = CausalInference(data, var_names, graph_gt, LinearRegression , discrete=False)

ate, y_treat,y_control = CausalInference_.ate(target, treatments)
print(f'Estimated ATE: {ate:.2f}')
toc = time.time()
print(f'{toc-tic:.2f}s')



Estimated ATE: 1.02

0.01s


### Условный средний экспериментальный эффект (CATE)

Данные генерируются с использованием следующей модели структурных уравнений:
$$C = "шум"(noise)$$
$$W = C + "шум"(noise)$$
$$X = C*W + "шум"(noise)$$
$$Y = C*X + "шум"(noise)$$

В нашем примере ниже мы будем рассматривать C как переменную условия, X как переменную вмешательства(влияющую переменную) и Y как целевую переменную. Шум, используемый в нашем примере, выбирается из стандартного распределения Гаусса.

In [5]:
T=2000
data, var_names, graph_gt = ConditionalDataGenerator(T=T, data_type='tabular', seed=0, discrete=False)
# var_names = ['C', 'W', 'X', 'Y']
treatment_var='X'
target = 'Y'
target_idx = var_names.index(target)


intervention1 = 0.1*np.ones(T, dtype=int)
intervention_data1,_,_ = ConditionalDataGenerator(T=T, data_type='tabular',\
                                    seed=0, intervention={treatment_var:intervention1}, discrete=False)

intervention2 = 0.9*np.ones(T, dtype=int)
intervention_data2,_,_ = ConditionalDataGenerator(T=T, data_type='tabular',\
                                    seed=0, intervention={treatment_var:intervention2}, discrete=False)
graph_gt

{'C': [], 'W': ['C'], 'X': ['C', 'W'], 'Y': ['C', 'X']}

In [6]:
condition_state=2.1
diff = np.abs(data[:,0] - condition_state)
idx = np.argmin(diff)
assert diff[idx]<0.1, f'No observational data exists for the conditional variable close to {condition_state}'


cate_gt = (intervention_data1[idx,target_idx] - intervention_data2[idx,target_idx])
print(f'True CATE: {cate_gt:.2f}')

####
treatments = define_treatments(treatment_var, intervention1,intervention2)
conditions = {'var_name': 'C', 'condition_value': condition_state}

tic = time.time()
model = partial(MLPRegressor, hidden_layer_sizes=(100,100), max_iter=200)
CausalInference_ = CausalInference(data, var_names, graph_gt, model, discrete=False)#

cate = CausalInference_.cate(target, treatments, conditions, model)
toc = time.time()
print(f'Estimated CATE: {cate:.2f}')
print(f'Time taken: {toc-tic:.2f}s')

True CATE: -1.69

Estimated CATE: -1.71

Time taken: 1.63s


## Дискретные данные

Процедура генерации синтетических данных для приведенных ниже примеров ATE и CATE идентична процедуре, описанной выше для непрерывного случая, за исключением того, что сгенерированные данные являются дискретными в приведенных ниже случаях.

### Средний экспериментальный эффект (ATE)
 Для этого примера мы будем использовать синтетические данные, которые имеют линейную зависимость между переменными данных.

In [7]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pickle as pkl
import time
from functools import partial

from causalai.data.data_generator import DataGenerator, ConditionalDataGenerator
from causalai.models.tabular.causal_inference import CausalInference
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

def define_treatments(name, t,c):
    treatment = dict(var_name=name,
                    treatment_value=t,
                    control_value=c)
    return treatment

In [8]:
fn = lambda x:x
coef = 0.1
sem = {
        'a': [], 
        'b': [('a', coef, fn), ('f', coef, fn)], 
        'c': [('b', coef, fn), ('f', coef, fn)],
        'd': [('b', coef, fn), ('b', coef, fn), ('g', coef, fn)],
        'e': [('f', coef, fn)], 
        'f': [],
        'g': [],
        }
T = 5000
data, var_names, graph_gt = DataGenerator(sem, T=T, seed=0, discrete=True, nstates=10)

graph_gt

{'a': [],
 'b': ['a', 'f'],
 'c': ['b', 'f'],
 'd': ['b', 'b', 'g'],
 'e': ['f'],
 'f': [],
 'g': []}

In [9]:

t1='a'
t2='b'
target = 'c'
target_var = var_names.index(target)

# note that states can be [0,1,...,9], so the multiples below must be in this range
intervention11 = 0*np.ones(T, dtype=int)
intervention21 = 9*np.ones(T, dtype=int)
intervention_data1,_,_ = DataGenerator(sem, T=T, seed=0,
                            intervention={t1: intervention11, t2:intervention21}, discrete=True, nstates=10)

intervention12 = 6*np.ones(T, dtype=int)
intervention22 = 5*np.ones(T, dtype=int)
intervention_data2,_,_ = DataGenerator(sem, T=T, seed=0,
                            intervention={t1:intervention12, t2:intervention22}, discrete=True, nstates=10)

true_effect = (intervention_data1[:,target_var] - intervention_data2[:,target_var]).mean()
print("Ground truth ATE = %.2f" %true_effect)

Ground truth ATE = 0.66


In [10]:

tic = time.time()

treatments = [define_treatments(t1, intervention11,intervention12),\
             define_treatments(t2, intervention21,intervention22)]
model = partial(MLPRegressor, hidden_layer_sizes=(100,100), max_iter=200) # LinearRegression
CausalInference_ = CausalInference(data, var_names, graph_gt, model, discrete=True)#
o, y_treat,y_control = CausalInference_.ate(target, treatments)
print(f'Estimated ATE: {o:.2f}')
toc = time.time()
print(f'Time taken: {toc-tic:.2f}s')


Estimated ATE: 0.50

Time taken: 0.98s


### CATE (conditional ATE)
For this example we will use synthetic data that has non-linear dependence among data variables.

In [11]:
T=5000
data, var_names, graph_gt = ConditionalDataGenerator(T=T, data_type='tabular', seed=0, discrete=True, nstates=10)
# var_names = ['C', 'W', 'X', 'Y']

treatment_var='X'
target = 'Y'
target_idx = var_names.index(target)

# note that states can be [0,1,...,9], so the multiples below must be in this range
intervention1 = 1*np.ones(T, dtype=int)
intervention_data1,_,_ = ConditionalDataGenerator(T=T, data_type='tabular',\
                                    seed=0, intervention={treatment_var:intervention1}, discrete=True, nstates=10)

intervention2 = 9*np.ones(T, dtype=int)
intervention_data2,_,_ = ConditionalDataGenerator(T=T, data_type='tabular',\
                                    seed=0, intervention={treatment_var:intervention2}, discrete=True, nstates=10)
graph_gt

{'C': [], 'W': ['C'], 'X': ['C', 'W'], 'Y': ['C', 'X']}

In [12]:
condition_var = 'C'
condition_var_idx = var_names.index(condition_var)
condition_state=1
idx = np.where(data[:,condition_var_idx]==condition_state)[0]
cate_gt = (intervention_data1[idx,target_idx] - intervention_data2[idx,target_idx]).mean()
print(f'True CATE: {cate_gt:.2f}')

####
treatments = define_treatments(treatment_var, intervention1,intervention2)
conditions = {'var_name': condition_var, 'condition_value': condition_state}

tic = time.time()
model = partial(MLPRegressor, hidden_layer_sizes=(100,100), max_iter=200)
CausalInference_ = CausalInference(data, var_names, graph_gt, model, discrete=True)

cate = CausalInference_.cate(target, treatments, conditions, model)
toc = time.time()
print(f'Estimated CATE: {cate:.2f}')
print(f'Time taken: {toc-tic:.2f}s')

True CATE: 6.01

Estimated CATE: 6.00

Time taken: 3.66s
